In [1]:
from code.visual import visualization as vs, word_clouds as wd
from code.preprocessing import load_data as ld, merge_df as mf
from code.computations import evaluation_w2v as ev, clustering as cl, dim_reduction as dm

C:\BA_thesis\BA_v2_31.03\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# basic
import os
import pandas as pd

# case_specific
import spacy
from sklearn.cluster import AgglomerativeClustering
from itertools import product
from gensim.models import Word2Vec
import hdbscan
import matplotlib.pyplot as plt

In [ ]:
os.chdir(r"C:/BA_thesis/BA_v2_31.03")
print(f"working directory: {os.getcwd()}")
input_path = os.getcwd() + '/files/corpus_data'

In [ ]:
""" 
1) load the data
"""
nlp = spacy.load("en_core_web_sm")
ld.load_data(dir_with_corpus_files=input_path,
             nlp=nlp,
             chunksize=40)

In [ ]:
"""
2) create corpus
"""
corpus = ld.TxtSubdirsCorpus("files/dfs")

In [ ]:
window = [1, 3]
epochs = [100]
sg = [0, 1]
vector_size = [100]

for w, e, s, v in product(window, epochs, sg, vector_size):
    print(f'Training of w{w}e{e}sg{s}v{v} haas started.')
    model = Word2Vec(
        sentences=corpus,
        window=w,
        min_count=5,
        epochs=e,
        sg=s,
        vector_size=v
    )
    model.save(f"files/models/w{w}e{e}sg{s}v{v}.model")